In [1]:
import requests
import os
from langchain.chains import APIChain
from langchain.chat_models import ChatOpenAI

In [2]:
os.environ["OPENAI_API_KEY"] = "API KEY"

In [3]:
llm = ChatOpenAI(temperature= 0, model_name="gpt-3.5-turbo", max_tokens = 256, verbose =True)

In [4]:
apiSpecs = """API Documentation
base_url: https://api.gbif.org/v1/
Endpoint: occurrence/search
Example API call: https://www.gbif.org/occurrence/search?q=Hemigrapsus%20nudus
"""

In [5]:
chain_new = APIChain.from_llm_and_api_docs(llm, apiSpecs, verbose=True)
response = chain_new.run('Please list the most recent occurrence with Tonicella lineata')
print(response)



> Entering new APIChain chain...
https://api.gbif.org/v1/occurrence/search?q=Tonicella%20lineata&limit=1&sort=occurrenceDate desc
{"offset":0,"limit":1,"endOfRecords":false,"count":457899,"results":[{"key":1989898085,"datasetKey":"965bbbd8-f762-11e1-a439-00145eb45e9a","publishingOrgKey":"c76cf030-2a95-11da-9cc1-b8a03c50a862","networkKeys":["17abcf75-2f1e-46dd-bf75-a5b21dd02655"],"installationKey":"60383ab8-f762-11e1-a439-00145eb45e9a","hostingOrganizationKey":"c76cf030-2a95-11da-9cc1-b8a03c50a862","publishingCountry":"DE","protocol":"BIOCASE","lastCrawled":"2023-07-14T22:44:48.042+00:00","lastParsed":"2023-07-15T07:20:55.169+00:00","crawlId":224,"extensions":{},"basisOfRecord":"PRESERVED_SPECIMEN","occurrenceStatus":"PRESENT","taxonKey":2306813,"kingdomKey":1,"phylumKey":52,"classKey":346,"orderKey":8278247,"familyKey":3246305,"genusKey":2306809,"speciesKey":2306813,"acceptedTaxonKey":2306813,"scientificName":"Tonicella lineata (Wood, 1815)","acceptedScientificName":"Tonicella lineat

In [23]:
chain_new = APIChain.from_llm_and_api_docs(llm, apiSpecs, verbose=True)
response = chain_new.run('Please list the first occurrence with Hemigrapsus nudus')
print(response)

Error in on_chain_start callback: 'name'


https://api.gbif.org/v1/occurrence/search?q=Hemigrapsus%20nudus&limit=1
{"offset":0,"limit":1,"endOfRecords":false,"count":40113,"results":[{"key":1065135676,"datasetKey":"005eb8d8-ed94-41be-89cf-e3115a9058e4","publishingOrgKey":"7b8aff00-a9f8-11d8-944b-b8a03c50a862","installationKey":"3b84ff5b-79ce-427c-acb7-b4004e160f66","hostingOrganizationKey":"7b8aff00-a9f8-11d8-944b-b8a03c50a862","publishingCountry":"US","protocol":"EML","lastCrawled":"2023-06-01T04:23:12.378+00:00","lastParsed":"2023-06-07T20:20:50.103+00:00","crawlId":310,"extensions":{},"basisOfRecord":"PRESERVED_SPECIMEN","individualCount":2,"occurrenceStatus":"PRESENT","taxonKey":2225773,"kingdomKey":1,"phylumKey":54,"classKey":229,"orderKey":637,"familyKey":11368965,"genusKey":2225769,"speciesKey":2225773,"acceptedTaxonKey":2225773,"scientificName":"Hemigrapsus nudus (Dana, 1851)","acceptedScientificName":"Hemigrapsus nudus (Dana, 1851)","kingdom":"Animalia","phylum":"Arthropoda","order":"Decapoda","family":"Varunidae","gen

In [2]:
import requests
import json

# GBIF API base URL
base_url = 'https://api.gbif.org/v1/'

# Function to search occurrences of a species
def search_species_occurrences(species_name, year = None):
    # GBIF Occurrence Search API endpoint
    url = f'{base_url}occurrence/search'

    # Parameters for the API request
    params = {
        'scientificName': species_name,
        'limit': 5 # Number of results to retrieve (adjust as needed)
    }
    
    if year:
        #Add year parameter if provided
        params["year"] = year
    
    try:
        # Set get request to GBIF API
        response = requests.get(url, params=params)
        response.raise_for_status()

        # Extract the repsonse data as JSON
        data = response.json()
        
        if "results" in data:
            results = data["results"]
            if results:
                occurrences = []
                for result in results:
#                     if all(attr in result for attr in ["key", "eventDate", "locality", "decimalLatitude", "decimalLongitude", "recordedBy", "kingdomKey","phylumKey", "waterBody"]):
                        occurrence_id = result.get("key", "N/A")
                        occurrence_sciName = result.get("scientificName", "N/A")
                        occurrence_record = result.get("recordedBy", "N/A")
                        occurrence_date = result.get("eventDate", "N/A")
                        occurrence_locality = result.get("locality", "N/A")
                        occurrence_bodywater = result.get("waterBody", "N/A")
                        occurrence_depth = result.get("depth", "N/A")
                        occurrence_elevation = result.get("elevation", "N/A")
                        occurrence_lat = result.get("decimalLatitude", "N/A")
                        occurrence_long = result.get("decimalLongitude", "N/A")
                        occurrence_kingdom = result.get("kingdomKey", "N/A")
                        occurrence_phylum = result.get("phylumKey", "N/A")
                

                        #Create a dictionary for the occurrences
                        occurrence = {
                            "occurrence_id": occurrence_id,
                            "occurrence_sciName": occurrence_sciName,
                            "occurrence_date": occurrence_date,
                            "location": occurrence_locality,
                            "latitude": occurrence_lat,
                            "longitude": occurrence_long,
                            "collected by": occurrence_record,
                            "kingdom": occurrence_kingdom,
                            "phylum": occurrence_phylum,
                            "body_water": occurrence_bodywater,
                            "depth": occurrence_depth,
                            "elevation": occurrence_elevation
                        }

                        #Add the occurrence to the list
                        occurrences.append(occurrence)
                return occurrences
            else:
                return []
        else:
            print("Error: Invalid reponse from GBIF API")
            return []
        
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")
    except requests.exceptions.RequestException as err:
        print(f"An error occurred: {err}")
          

In [3]:
# Example usage
species_name = "Tonicella lineata"  # Replace with your desired species name
year = 2014 #replace with year of article
occurrences = search_species_occurrences(species_name, year)

In [4]:
# Print or process the occurrences as needed
for occurrence in occurrences:
    print(f"Occurrence ID: {occurrence['occurrence_id']}")
    print(f"Scientific Name: {occurrence['occurrence_sciName']}")
    print(f"Occurrence Date: {occurrence['occurrence_date']}")
    print(f"Recorded by: {occurrence['collected by']}")
    print(f"Location: {occurrence['location']}")
    print(f"Depth: {occurrence['depth']}")
    print(f"Elevation: {occurrence['elevation']}")
    print(f"Body of Water: {occurrence['body_water']}")
    print(f"Latitude: {occurrence['latitude']}")
    print(f"Longitude: {occurrence['longitude']}")
    print(f"Kingdom: {occurrence['kingdom']}")
    print(f"Phylum: {occurrence['phylum']}")
    print("-" * 20)

Occurrence ID: 4039335302
Scientific Name: Tonicella lineata (Wood, 1815)
Occurrence Date: 2014-01-11T13:38:33
Recorded by: Kris Liang
Location: N/A
Depth: N/A
Elevation: N/A
Body of Water: N/A
Latitude: 37.520271
Longitude: -122.515915
Kingdom: 1
Phylum: 52
--------------------
Occurrence ID: 3357460927
Scientific Name: Tonicella lineata (Wood, 1815)
Occurrence Date: 2014-01-03T00:00:00
Recorded by: Andy Lamb
Location: Baines Point
Depth: N/A
Elevation: N/A
Body of Water: N/A
Latitude: 49.007465
Longitude: -123.587784
Kingdom: 1
Phylum: 52
--------------------
Occurrence ID: 891052717
Scientific Name: Tonicella lineata (Wood, 1815)
Occurrence Date: 2014-02-27T15:09:28
Recorded by: Liam O'Brien
Location: N/A
Depth: N/A
Elevation: N/A
Body of Water: N/A
Latitude: 37.494351
Longitude: -122.498529
Kingdom: 1
Phylum: 52
--------------------
Occurrence ID: 1837784847
Scientific Name: Tonicella lineata (Wood, 1815)
Occurrence Date: 2014-03-23T12:03:00
Recorded by: Donna Pomeroy
Location: N/A